In [ ]:
    # !pip install surprise

In [ ]:
#importing relevant libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.subplots as sp
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import skew
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score

from surprise import SVDpp
from sklearn.metrics import mean_squared_error
from surprise import KNNBasic
from surprise import SVD
from surprise import Reader,Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from sklearn.pipeline import Pipeline


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
new_data = pd.read_csv('/content/drive/MyDrive/new-data.csv')

In [ ]:
new_data = new_data.drop(['Unnamed: 0'], axis=1)

In [ ]:
new_data.shape

(630808, 21)

In [ ]:
new_data.columns

Index(['author_id', 'product_id', 'product_name_x', 'brand_name_x',
       'loves_count', 'rating_x', 'reviews', 'size', 'ingredients',
       'price_usd_x', 'highlights', 'primary_category', 'secondary_category',
       'tertiary_category', 'is_recommended', 'helpfulness', 'review_text',
       'skin_tone', 'eye_color', 'skin_type', 'hair_color'],
      dtype='object')

In [ ]:
new_data.secondary_category.value_counts()

Moisturizers       195305
Treatments         165198
Cleansers          133061
Eye Care            48481
Masks               42314
Sunscreen           31010
Self Tanners         8652
Wellness             5828
High Tech Tools       959
Name: secondary_category, dtype: int64

In [ ]:
new_data.tertiary_category.value_counts()

Moisturizers                 140309
Face Serums                  135560
Face Wash & Cleansers         79961
Eye Creams & Treatments       44685
Face Masks                    40414
Face Sunscreen                30276
Toners                        25168
Mists & Essences              22099
Face Oils                     20734
Facial Peels                  19634
Exfoliators                   17061
Blemish & Acne Treatments     10004
Makeup Removers                7516
BB & CC Creams                 6862
For Face                       5358
Beauty Supplements             5254
Night Creams                   4662
Eye Masks                      3796
Face Wipes                     3355
For Body                       3294
Sheet Masks                    1900
Teeth Whitening                 762
Body Sunscreen                  734
Decollete & Neck Creams         639
Holistic Wellness               338
Facial Rollers                  236
Anti-Aging                      197
Name: tertiary_category, dty

In [ ]:
len(new_data.tertiary_category.value_counts())

27

In [ ]:
new_data.price_usd_x.describe()

count    630808.000000
mean         54.049957
std          41.285970
min           3.000000
25%          32.000000
50%          46.000000
75%          68.000000
max         425.000000
Name: price_usd_x, dtype: float64

In [ ]:
new_data.highlights[5000]

"['Hyaluronic Acid', 'Best for Dry Skin', 'Good for: Loss of firmness', 'Clean + Planet Positive', 'Good for: Anti-Aging', 'Cruelty-Free']"

### Preprocessing

In [ ]:
# Engineer the price_range column
def engineer_price_range(price):
    if price < 32:
        return 'low'
    elif price < 68:
        return 'medium'
    else:
        return 'high'

new_data['price_range'] = new_data['price_usd_x'].apply(engineer_price_range)

### Modeling

In [ ]:
# Generate recommendations based on user input
def generate_recommendation(price_range, tertiary_category, skin_type, skin_tone, model):
    # Filter products based on user input criteria
    filtered_data = new_data[
        (new_data['price_range'] == price_range) &
        (new_data['tertiary_category'] == tertiary_category) &
        (new_data['skin_type'] == skin_type) &
        (new_data['skin_tone'] == skin_tone)
    ]
   # Select a random product from the filtered data
    if not filtered_data.empty:
        product = filtered_data.sample(n=1)
        product_id = product['product_id'].iloc[0]

        # Generate prediction for the selected product
        prediction = model.predict('new_user', product_id)

        # Return product name and ingredients
        product_name = product['product_name_x'].iloc[0]
        ingredients = product['ingredients'].iloc[0]

        return product_name, ingredients

    return None, None

### KNNBasic- Baseline Model

In [ ]:
sample_data=new_data.sample(n=20000, random_state=42)

In [ ]:
# Prepare the data for Surprise library
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(sample_data[['author_id', 'product_id', 'rating_x']], reader)
trainset1, testset1 = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
model1 = KNNBasic(random_state=42)
model1.fit(trainset1)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# Evaluate the recommendation model
predictions = model1.test(testset1)

# Calculate RMSE score
rmse = accuracy.rmse(predictions)
print("RMSE:", rmse)

RMSE: 0.2725
RMSE: 0.27252509097857475


In [ ]:
# Example usage
price_range ='medium'
tertiary_category = 'Toners'
skin_type = 'oily'
skin_tone = 'light'

product_name, ingredients = generate_recommendation(price_range, tertiary_category, skin_type, skin_tone, model1)

if product_name is not None:
    print("Recommended Product:", product_name)
    print("Ingredients:", ingredients)
else:
    print("No recommendation found for the given input criteria.")

Recommended Product: Ready Steady Glow Daily AHA Tonic
Ingredients: ['Aqua (Water), Sodium Lactate, Lactic Acid, Heptyl Glucoside, Potassium Azeloyl Diglycinate, Triethyl Citrate, Glyceryl Caprylate, Salix Nigra (Willow) Bark Extract, Benzoic Acid, Citrus Nobilis (Mandarin) Peel Oil, Citrus Aurantium Dulcis (Orange) Flower Oil, Citrus Grandis (Grapefruit) Peel Oil, Citrus Tangerina (Tangerine) Peel Oil, Heptanol, Parfum (Fragrance), Limonene.']


### SVD

In [ ]:
# !pip install surprise

In [ ]:
# Prepare the data for Surprise library
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(new_data[['author_id', 'product_id', 'rating_x']], reader)
trainset2, testset2 = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Build and train the recommendation model
model2 = SVD(random_state=42)
model2.fit(trainset2)

In [ ]:
# Evaluate the recommendation model
predictions2 = model2.test(testset2)

# Calculate RMSE score
rmse = accuracy.rmse(predictions2)
print("RMSE:", rmse)

RMSE: 0.0338
RMSE: 0.03378027216861353


In [ ]:
# Generate recommendations based on user input
def generate_recommendation(price_range, tertiary_category, skin_type, skin_tone, model):
    # Filter products based on user input criteria
    filtered_data = new_data[
        (new_data['price_range'] == price_range) &
        (new_data['tertiary_category'] == tertiary_category) &
        (new_data['skin_type'] == skin_type) &
        (new_data['skin_tone'] == skin_tone)
    ]
   # Select a random product from the filtered data
    if not filtered_data.empty:
        sorted_data = filtered_data.sort_values(by='rating_x', ascending=False)
        product = sorted_data.sample(n=1)
        product_id = product['product_id'].iloc[0]

        # Generate prediction for the selected product
        prediction = model.predict('new_user', product_id)

    #     # Return product name and ingredients
    #     product_name = product['product_name_x'].iloc[0]
    #     ingredients = product['ingredients'].iloc[0]

    #     return product_name, ingredients
    # return None, None

In [ ]:
# Example usage
price_range ='medium'
tertiary_category = 'Toners'
skin_type = 'oily'
skin_tone = 'light'

# product_name, ingredients = generate_recommendation(price_range, tertiary_category, skin_type, skin_tone, model2)
prediction = generate_recommendation(price_range, tertiary_category, skin_type, skin_tone, model2)
prediction

# if product_name is not None:
#     print("Recommended Product:", product_name)
#     print("Ingredients:", ingredients)
# else:
#     print("No recommendation found for the given input criteria.")

### SVDpp

In [ ]:
# Build and train the recommendation model
model3 = SVDpp(random_state=42)
model3.fit(trainset2)# Build and train the recommendation model

In [ ]:
# Evaluate the recommendation model
predictions3 = model3.test(testset2)

# Calculate RMSE score
rmse = accuracy.rmse(predictions3)
print("RMSE:", rmse)

RMSE: 0.0275
RMSE: 0.02745804616958275


In [ ]:
# Example usage
price_range ='medium'
tertiary_category = 'Toners'
skin_type = 'oily'
skin_tone = 'light'

product_name, ingredients = generate_recommendation(price_range, tertiary_category, skin_type, skin_tone, model3)

if product_name is not None:
    print("Recommended Product:", product_name)
    print("Ingredients:", ingredients)
else:
    print("No recommendation found for the given input criteria.")

TypeError: ignored

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(new_data[["author_id", "product_id", "rating_x"]], reader)

pipeline = Pipeline([
    ('reader', reader),
    ('load_data', data),
    ('train_test_split', train_test_split(data, test_size=0.2, random_state=42)),
    ('model', SVD(random_state=42,n_epochs=20, n_factors=1, biased=True, lr_all=0.005, reg_all=0.06))
])


### Hyperparameter tuning

In [ ]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

# Load the dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(new_data[["author_id", "product_id", "rating_x"]], reader)

# Define the parameter grid
param_grid = {'n_epochs': [10, 20, 30], 'n_factors': [50, 100, 150], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.02, 0.04, 0.06]}

# Perform grid search using cross-validation
best_score = float('inf')
best_params = {}

for n_epochs in param_grid['n_epochs']:
    for n_factors in param_grid['n_factors']:
        for lr_all in param_grid['lr_all']:
            for reg_all in param_grid['reg_all']:
                # Create and evaluate the SVD model with the current parameters
                model = SVD(n_epochs=n_epochs, n_factors=n_factors, lr_all=lr_all, reg_all=reg_all, random_state=42)
                results = cross_validate(model, data, measures=['RMSE'], cv=3, verbose=False)

                # Compute average RMSE score
                avg_rmse = sum(results['test_rmse']) / len(results['test_rmse'])

                # Check if current parameters yield a better score
                if avg_rmse < best_score:
                    best_score = avg_rmse
                    best_params = {'n_epochs': n_epochs, 'n_factors': n_factors, 'lr_all': lr_all, 'reg_all': reg_all}

# Print the best RMSE score and the corresponding parameters
print("Best RMSE score:", best_score)
print("Best parameters:", best_params)


Best RMSE score: 0.019928819420548884
Best parameters: {'n_epochs': 30, 'n_factors': 50, 'lr_all': 0.01, 'reg_all': 0.02}


In [ ]:
# ('model', SVD(n_epochs=20, n_factors=1, biased=True, lr_all=0.005, reg_all=0.06))

In [ ]:
# import pickle


In [ ]:
# import pickle
# # Save the model using pickle
# with open('svdpp_model.pkl', 'wb') as f:
#     pickle.dump(model3, f)

# # Load the model from the pickle file
# with open('svdpp_model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)